In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [6]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
ml=["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]
for i in ml:
    df[i].replace(0,np.nan,inplace=True)
    df[i].fillna(df[i].mean(),inplace=True)

In [8]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.435949,12.096346,8.790942,85.021108,6.875151,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.750000,64.000000,25.000000,121.500000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.202592,29.153420,155.548223,32.400000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,155.548223,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
x=df.iloc[:,1:-1]
x

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,148.0,72.0,35.00000,155.548223,33.6,0.627,50
1,85.0,66.0,29.00000,155.548223,26.6,0.351,31
2,183.0,64.0,29.15342,155.548223,23.3,0.672,32
3,89.0,66.0,23.00000,94.000000,28.1,0.167,21
4,137.0,40.0,35.00000,168.000000,43.1,2.288,33
...,...,...,...,...,...,...,...
763,101.0,76.0,48.00000,180.000000,32.9,0.171,63
764,122.0,70.0,27.00000,155.548223,36.8,0.340,27
765,121.0,72.0,23.00000,112.000000,26.2,0.245,30
766,126.0,60.0,29.15342,155.548223,30.1,0.349,47


In [10]:
y=df["Outcome"]
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [11]:
#standarization
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [12]:
# Applying train test split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.20,random_state=1)

In [13]:
def mymodel(model):
    model.fit(xtrain,ytrain)
    ypred=model.predict(xtest)
    print(classification_report(ytest,ypred))

In [14]:
lreg=LogisticRegression()
knn=KNeighborsClassifier()
svm=SVC()
dt=DecisionTreeClassifier()

In [15]:
mymodel(lreg)

              precision    recall  f1-score   support

           0       0.80      0.89      0.84        99
           1       0.75      0.60      0.67        55

    accuracy                           0.79       154
   macro avg       0.78      0.74      0.75       154
weighted avg       0.78      0.79      0.78       154



In [16]:
mymodel(knn)

              precision    recall  f1-score   support

           0       0.80      0.87      0.83        99
           1       0.72      0.62      0.67        55

    accuracy                           0.78       154
   macro avg       0.76      0.74      0.75       154
weighted avg       0.78      0.78      0.77       154



In [17]:
mymodel(svm)

              precision    recall  f1-score   support

           0       0.79      0.93      0.86        99
           1       0.82      0.56      0.67        55

    accuracy                           0.80       154
   macro avg       0.80      0.75      0.76       154
weighted avg       0.80      0.80      0.79       154



In [18]:
mymodel(dt)

              precision    recall  f1-score   support

           0       0.77      0.82      0.79        99
           1       0.63      0.56      0.60        55

    accuracy                           0.73       154
   macro avg       0.70      0.69      0.70       154
weighted avg       0.72      0.73      0.72       154



In [19]:
dt.score(xtrain,ytrain)

1.0

# HYPER-PARAMETER

In [20]:
dt1=DecisionTreeClassifier(max_depth=5)
mymodel(dt1)

              precision    recall  f1-score   support

           0       0.81      0.84      0.83        99
           1       0.69      0.65      0.67        55

    accuracy                           0.77       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.77      0.77      0.77       154



In [21]:
for i in range(1,51):
    dt2=DecisionTreeClassifier(max_depth=i)
    dt2.fit(xtrain,ytrain)
    ypred=dt2.predict(xtest)
    print(f"{i} = {accuracy_score(ytest,ypred)}")

1 = 0.7337662337662337
2 = 0.7922077922077922
3 = 0.7792207792207793
4 = 0.7662337662337663
5 = 0.7662337662337663
6 = 0.8116883116883117
7 = 0.7532467532467533
8 = 0.7272727272727273
9 = 0.7207792207792207
10 = 0.6818181818181818
11 = 0.7077922077922078
12 = 0.7012987012987013
13 = 0.6818181818181818
14 = 0.7012987012987013
15 = 0.6948051948051948
16 = 0.6753246753246753
17 = 0.7337662337662337
18 = 0.6883116883116883
19 = 0.7142857142857143
20 = 0.6753246753246753
21 = 0.6883116883116883
22 = 0.6818181818181818
23 = 0.7272727272727273
24 = 0.7012987012987013
25 = 0.7142857142857143
26 = 0.6883116883116883
27 = 0.6948051948051948
28 = 0.6948051948051948
29 = 0.6818181818181818
30 = 0.7077922077922078
31 = 0.7142857142857143
32 = 0.6688311688311688
33 = 0.7012987012987013
34 = 0.7077922077922078
35 = 0.6688311688311688
36 = 0.7142857142857143
37 = 0.7142857142857143
38 = 0.6883116883116883
39 = 0.7207792207792207
40 = 0.6948051948051948
41 = 0.7077922077922078
42 = 0.6883116883116883
4

In [22]:
#best value of max_depth=6
dt3=DecisionTreeClassifier(max_depth=6)
mymodel(dt3)

              precision    recall  f1-score   support

           0       0.88      0.78      0.82        99
           1       0.67      0.80      0.73        55

    accuracy                           0.79       154
   macro avg       0.77      0.79      0.78       154
weighted avg       0.80      0.79      0.79       154



In [23]:
dt4=DecisionTreeClassifier(min_samples_leaf=10)
mymodel(dt4)

              precision    recall  f1-score   support

           0       0.85      0.89      0.87        99
           1       0.78      0.73      0.75        55

    accuracy                           0.83       154
   macro avg       0.82      0.81      0.81       154
weighted avg       0.83      0.83      0.83       154



In [24]:
for i in range(1,75):
    dt5=DecisionTreeClassifier(min_samples_leaf=i)
    dt5.fit(xtrain,ytrain)
    ypred=dt5.predict(xtest)
    print(f"{i} = {accuracy_score(ytest,ypred)}")

1 = 0.6948051948051948
2 = 0.7272727272727273
3 = 0.6753246753246753
4 = 0.6883116883116883
5 = 0.7077922077922078
6 = 0.7597402597402597
7 = 0.7987012987012987
8 = 0.7987012987012987
9 = 0.7857142857142857
10 = 0.8311688311688312
11 = 0.8311688311688312
12 = 0.8311688311688312
13 = 0.8311688311688312
14 = 0.8311688311688312
15 = 0.8181818181818182
16 = 0.8116883116883117
17 = 0.8116883116883117
18 = 0.8116883116883117
19 = 0.8376623376623377
20 = 0.8376623376623377
21 = 0.8376623376623377
22 = 0.8246753246753247
23 = 0.8246753246753247
24 = 0.8246753246753247
25 = 0.8246753246753247
26 = 0.8246753246753247
27 = 0.8246753246753247
28 = 0.8181818181818182
29 = 0.7987012987012987
30 = 0.8051948051948052
31 = 0.8051948051948052
32 = 0.8246753246753247
33 = 0.8246753246753247
34 = 0.8246753246753247
35 = 0.8246753246753247
36 = 0.8246753246753247
37 = 0.8246753246753247
38 = 0.8051948051948052
39 = 0.8051948051948052
40 = 0.8051948051948052
41 = 0.7922077922077922
42 = 0.7922077922077922
4

In [25]:
dt6=DecisionTreeClassifier(min_samples_leaf=10)
mymodel(dt6)

              precision    recall  f1-score   support

           0       0.85      0.89      0.87        99
           1       0.78      0.73      0.75        55

    accuracy                           0.83       154
   macro avg       0.82      0.81      0.81       154
weighted avg       0.83      0.83      0.83       154



In [26]:
dt7=DecisionTreeClassifier(max_depth=6,min_samples_leaf=10)
mymodel(dt7)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88        99
           1       0.77      0.80      0.79        55

    accuracy                           0.84       154
   macro avg       0.83      0.83      0.83       154
weighted avg       0.85      0.84      0.84       154



# GINI

In [27]:
dt8=DecisionTreeClassifier(criterion="gini",min_samples_leaf=10)
mymodel(dt8)

              precision    recall  f1-score   support

           0       0.85      0.89      0.87        99
           1       0.78      0.73      0.75        55

    accuracy                           0.83       154
   macro avg       0.82      0.81      0.81       154
weighted avg       0.83      0.83      0.83       154



In [28]:
dt9=DecisionTreeClassifier(criterion="gini",max_depth=6)
mymodel(dt9)

              precision    recall  f1-score   support

           0       0.86      0.80      0.83        99
           1       0.68      0.76      0.72        55

    accuracy                           0.79       154
   macro avg       0.77      0.78      0.77       154
weighted avg       0.79      0.79      0.79       154



In [29]:
dt10=DecisionTreeClassifier(criterion="entropy",max_depth=6)
mymodel(dt10)

              precision    recall  f1-score   support

           0       0.87      0.78      0.82        99
           1       0.66      0.78      0.72        55

    accuracy                           0.78       154
   macro avg       0.76      0.78      0.77       154
weighted avg       0.79      0.78      0.78       154



In [30]:
for i in range(1,50):
    dt11=DecisionTreeClassifier(criterion="entropy",max_depth=i)
    dt11.fit(xtrain,ytrain)
    ypred=dt11.predict(xtest)
    print(f"{i}= {accuracy_score(ytest,ypred)}")

1= 0.7337662337662337
2= 0.7337662337662337
3= 0.7922077922077922
4= 0.7857142857142857
5= 0.7857142857142857
6= 0.7662337662337663
7= 0.7662337662337663
8= 0.7532467532467533
9= 0.7337662337662337
10= 0.7467532467532467
11= 0.7402597402597403
12= 0.7402597402597403
13= 0.7337662337662337
14= 0.7597402597402597
15= 0.7337662337662337
16= 0.7337662337662337
17= 0.7402597402597403
18= 0.7337662337662337
19= 0.7337662337662337
20= 0.7142857142857143
21= 0.7207792207792207
22= 0.7272727272727273
23= 0.7337662337662337
24= 0.7402597402597403
25= 0.7207792207792207
26= 0.7272727272727273
27= 0.7077922077922078
28= 0.7012987012987013
29= 0.7207792207792207
30= 0.7337662337662337
31= 0.7402597402597403
32= 0.7207792207792207
33= 0.7142857142857143
34= 0.7207792207792207
35= 0.7142857142857143
36= 0.7272727272727273
37= 0.7142857142857143
38= 0.7142857142857143
39= 0.7337662337662337
40= 0.7337662337662337
41= 0.7207792207792207
42= 0.7207792207792207
43= 0.7077922077922078
44= 0.71428571428571

In [31]:
#best value of max_depth when criterion=entropy
dt12=DecisionTreeClassifier(criterion="entropy",max_depth=5)
mymodel(dt12)

              precision    recall  f1-score   support

           0       0.88      0.75      0.81        99
           1       0.64      0.82      0.72        55

    accuracy                           0.77       154
   macro avg       0.76      0.78      0.76       154
weighted avg       0.80      0.77      0.78       154



In [32]:
dt13=DecisionTreeClassifier(criterion="entropy",min_samples_leaf=10)
mymodel(dt13)

              precision    recall  f1-score   support

           0       0.79      0.85      0.82        99
           1       0.69      0.60      0.64        55

    accuracy                           0.76       154
   macro avg       0.74      0.72      0.73       154
weighted avg       0.75      0.76      0.76       154



In [33]:
for i in range(1,50):
    dt14=DecisionTreeClassifier(criterion="entropy",min_samples_leaf=i)
    dt14.fit(xtrain,ytrain)
    ypred=dt14.predict(xtest)
    print(f"{i}= {accuracy_score(ytest,ypred)}")

1= 0.7142857142857143
2= 0.7142857142857143
3= 0.7142857142857143
4= 0.7727272727272727
5= 0.7597402597402597
6= 0.7272727272727273
7= 0.7402597402597403
8= 0.7922077922077922
9= 0.7597402597402597
10= 0.7792207792207793
11= 0.7922077922077922
12= 0.7922077922077922
13= 0.7987012987012987
14= 0.8181818181818182
15= 0.8116883116883117
16= 0.8116883116883117
17= 0.8116883116883117
18= 0.8116883116883117
19= 0.8116883116883117
20= 0.8116883116883117
21= 0.8051948051948052
22= 0.8051948051948052
23= 0.8051948051948052
24= 0.8181818181818182
25= 0.8181818181818182
26= 0.8051948051948052
27= 0.8311688311688312
28= 0.8246753246753247
29= 0.8051948051948052
30= 0.8116883116883117
31= 0.8116883116883117
32= 0.8181818181818182
33= 0.8181818181818182
34= 0.8181818181818182
35= 0.8181818181818182
36= 0.8181818181818182
37= 0.8181818181818182
38= 0.8181818181818182
39= 0.8181818181818182
40= 0.8181818181818182
41= 0.8181818181818182
42= 0.8181818181818182
43= 0.7987012987012987
44= 0.79870129870129

In [34]:
# best value of min_samples_leaf=15 when criterion=entropy
dt15=DecisionTreeClassifier(criterion="entropy",min_samples_leaf=14)
mymodel(dt15)

              precision    recall  f1-score   support

           0       0.84      0.89      0.86        99
           1       0.78      0.69      0.73        55

    accuracy                           0.82       154
   macro avg       0.81      0.79      0.80       154
weighted avg       0.82      0.82      0.82       154



# final models

In [35]:
#gini
dt16=DecisionTreeClassifier(max_depth=6,min_samples_leaf=14)
mymodel(dt16)

              precision    recall  f1-score   support

           0       0.90      0.86      0.88        99
           1       0.77      0.84      0.80        55

    accuracy                           0.85       154
   macro avg       0.84      0.85      0.84       154
weighted avg       0.86      0.85      0.85       154



In [36]:
#entropy
dt17=DecisionTreeClassifier(criterion="entropy",min_samples_leaf=14)
mymodel(dt17)

              precision    recall  f1-score   support

           0       0.84      0.89      0.86        99
           1       0.78      0.69      0.73        55

    accuracy                           0.82       154
   macro avg       0.81      0.79      0.80       154
weighted avg       0.82      0.82      0.82       154

